# Phase:1 Data Extraction

Install the spotipy library using the pip command<br>
&emsp;1.pip install spotipy<br>
Enter the spotify client_id and client_secret of your account to access the Api.<br>
&emsp;2.Refer : https://developer.spotify.com/dashboard/<br>
&emsp;&ensp;&ensp;Sign in with your account and create an app.<br>
&emsp;&ensp;&ensp;enter the app to access your client_id and client_secret<br>

In [49]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
client_id = "f35d6bd83a4040c4929dc260be638d34"
client_secret = "6aa70b43e4d04ef2885df7b5f64837eb"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 
import requests
import pandas as pd
import time 

Create a connection to the Api

In [50]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']


In [51]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

After sucessfull connection enter you favourite artist in the artist_id<br>
&emsp;right click artist profile-->share link --> copy link to artist <br>
&emsp;Example : https://open.spotify.com/artist/0C8ZW7ezQVs4URX5aX7Kqx?si=0b1c0f5d36014512<br>
&emsp;Here 0C8ZW7ezQVs4URX5aX7Kqx is the artist id for selena gomez<br>

In [65]:
BASE_URL = 'https://api.spotify.com/v1/'

# your artist ID - The Weeknd
artist_id = '7jVv8c5Fj3E9VhNjxT4snq'

# extract all albums
r = requests.get(BASE_URL + 'artists/' + artist_id + '/albums', 
                 headers=headers, 
                 params={'include_groups': 'album', 'limit': 50})
d = r.json()
print(d)


{'href': 'https://api.spotify.com/v1/artists/7jVv8c5Fj3E9VhNjxT4snq/albums?offset=0&limit=50&include_groups=album', 'items': [{'album_group': 'album', 'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7jVv8c5Fj3E9VhNjxT4snq'}, 'href': 'https://api.spotify.com/v1/artists/7jVv8c5Fj3E9VhNjxT4snq', 'id': '7jVv8c5Fj3E9VhNjxT4snq', 'name': 'Lil Nas X', 'type': 'artist', 'uri': 'spotify:artist:7jVv8c5Fj3E9VhNjxT4snq'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IQ', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM

Extract the data from the api response into a list

In [66]:
audiofeature = []   # will hold all audio feature info
albums = [] # to keep track of duplicates

# loop over albums and get all tracks
for album in d['items']:
    album_name = album['name']

    # here's a hacky way to skip over albums we've already grabbed
    trim_name = album_name.split('(')[0].strip()
    if trim_name.upper() in albums:
        continue
    albums.append(trim_name.upper()) # use upper() to standardize
    
    # this takes a few seconds so let's keep track of progress    
    print(album_name)
    
    # pull all tracks from this album
    r = requests.get(BASE_URL + 'albums/' + album['id'] + '/tracks', 
        headers=headers)
    tracks = r.json()['items']
    
    for track in tracks:
        # get audio features (key, liveness, danceability, ...)
        f = requests.get(BASE_URL + 'audio-features/' + track['id'], 
            headers=headers)
        f = f.json()
        
        # combine with album info
        f.update({
            'track_name': track['name'],
            'album_name': album_name,
            'release_date': album['release_date'],
            'album_id': album['id']
        })
        
        audiofeature.append(f)

MONTERO


In [67]:
'''

Create a dataFrame with the list

'''

df2 = pd.DataFrame(audiofeature)
df2.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,album_name,release_date,album_id
0,0.593,0.503,8,-6.725,0,0.2200,0.29300,0.000000,0.4050,0.710,...,1SC5rEoYDGUK4NfG82494W,spotify:track:1SC5rEoYDGUK4NfG82494W,https://api.spotify.com/v1/tracks/1SC5rEoYDGUK...,https://api.spotify.com/v1/audio-analysis/1SC5...,137704,4,MONTERO (Call Me By Your Name),MONTERO,2021-09-17,6pOiDiuDQqrmo5DbG0ZubR
1,0.685,0.717,0,-6.456,0,0.0411,0.10100,0.000019,0.2660,0.549,...,3grVoZ10bm2jUGpo7BxpuF,spotify:track:3grVoZ10bm2jUGpo7BxpuF,https://api.spotify.com/v1/tracks/3grVoZ10bm2j...,https://api.spotify.com/v1/audio-analysis/3grV...,221203,4,DEAD RIGHT NOW,MONTERO,2021-09-17,6pOiDiuDQqrmo5DbG0ZubR
2,0.741,0.691,10,-7.395,0,0.0672,0.02210,0.000000,0.0476,0.892,...,5Z9KJZvQzH6PFmb8SNkxuk,spotify:track:5Z9KJZvQzH6PFmb8SNkxuk,https://api.spotify.com/v1/tracks/5Z9KJZvQzH6P...,https://api.spotify.com/v1/audio-analysis/5Z9K...,212353,4,INDUSTRY BABY (feat. Jack Harlow),MONTERO,2021-09-17,6pOiDiuDQqrmo5DbG0ZubR
3,0.737,0.846,1,-4.510,0,0.2200,0.00614,0.000000,0.0486,0.546,...,0e8nrvls4Qqv5Rfa2UhqmO,spotify:track:0e8nrvls4Qqv5Rfa2UhqmO,https://api.spotify.com/v1/tracks/0e8nrvls4Qqv...,https://api.spotify.com/v1/audio-analysis/0e8n...,143901,4,THATS WHAT I WANT,MONTERO,2021-09-17,6pOiDiuDQqrmo5DbG0ZubR
4,0.519,0.464,10,-18.957,0,0.5600,0.90500,0.000000,0.4480,0.287,...,1BMu5TWvalAGpi5jlhFEBb,spotify:track:1BMu5TWvalAGpi5jlhFEBb,https://api.spotify.com/v1/tracks/1BMu5TWvalAG...,https://api.spotify.com/v1/audio-analysis/1BMu...,24006,4,THE ART OF REALIZATION,MONTERO,2021-09-17,6pOiDiuDQqrmo5DbG0ZubR


In [68]:
'''

Extract the required columns alone to another DataFrame

'''

df3 = pd.DataFrame(data=df2[['id','track_name', 'album_name','album_id','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']],columns=['id','track_name', 'album_name','album_id','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'],)

In [69]:
df3['artist_id'] = '7jVv8c5Fj3E9VhNjxT4snq'
df3['artist_name'] = 'lil nas x'

In [70]:
'''

Write the DataFrame into the same directory as your .ipynb file as a csv file with index set to false

'''
import os 
df3.to_csv('music_lilnasx.csv',index=False)